***AVISO**: Esse Notebook foi feito com base na estrutura do Databricks Free Edition, que utiliza catálogos.*

# **ETAPA 5 - ANÁLISES AD-HOC**

---
---

<br>

Essa etapa será responsável por responder perguntas pré-selecionadas que são ad-hoc's, utilizando resultado gerado pela estrutura completa das etapas anteriores.

*`Complete as informações necessárias nos trechos que estão destacados em vermelho assim como esse, seguindo o padrão snake_case.`*

<br><br>

---
---

### Pergunta de Negócio 01 - **_Como evoluíram as vendas mensais por categoria entre 2022 e 2025?_**

> **CONSULTA:** Agregação mensal (ano/mes) de valor_total por categoria_id/categoria_nome.

> **INSIGHT ESPERADO:** Identificar sazonalidade e categorias com maior crescimento relativo.

A seguir, código que executa consulta com cenário da pergunta:

In [0]:
from pyspark.sql.functions import sum, col, desc, row_number, round
import matplotlib.pyplot as plt
from pyspark.sql.window import Window
from pyspark.sql.functions import lit

# Tabela fato_vendas 
fato_vendas = "dataexperts.gold.fato_vendas"
dim_categoria  = "dataexperts.gold.dim_categoria_produto"

# Seleção da categoria e de sua respectiva sk
categoria_produto = (
    spark.table(dim_categoria)
         .select("categoria_nome", "sk_categoria")
         .distinct()
)

# Seleção de vendas totais por categoria em função de ano/mes
vendas = (spark.table(fato_vendas)
                    .groupBy("ano", "mes", "sk_categoria")
                    .agg(sum("valor_total").alias("total_vendas"))  
                )

vendas = (vendas.join(categoria_produto.select("sk_categoria", "categoria_nome"), on="sk_categoria").drop("sk_categoria"))

# Separacao por categoria
for categoria in categoria_produto.select("categoria_nome").collect():
    vendas = vendas.withColumn(
        "mes_num",
        when(col("mes") == "Janeiro", lit(1))
        .when(col("mes") == "Fevereiro", lit(2))
        .when(col("mes") == "Março", lit(3))
        .when(col("mes") == "Abril", lit(4))
        .when(col("mes") == "Maio", lit(5))
        .when(col("mes") == "Junho", lit(6))
        .when(col("mes") == "Julho", lit(7))
        .when(col("mes") == "Agosto", lit(8))
        .when(col("mes") == "Setembro", lit(9))
        .when(col("mes") == "Outubro", lit(10))
        .when(col("mes") == "Novembro", lit(11))
        .otherwise(lit(12))
    )

    vendas_categoria = (
        vendas
            .filter(col("categoria_nome") == categoria.categoria_nome)
            .orderBy("ano", "mes_num")
            .select("ano", "mes", "total_vendas")
            .collect()
    )
    
    if not vendas_categoria:
        continue

    x = [f"{r.ano}-{str(r.mes).zfill(2)}" for r in vendas_categoria]
    y = [r.total_vendas for r in vendas_categoria]

    # Plot
    plt.figure(figsize=(18, 4))
    plt.plot(x, y, marker="o")
    plt.title(f"Vendas - {categoria.categoria_nome}")
    plt.xlabel("Período (Ano-Mês)")
    plt.ylabel("Total de Vendas")
    plt.xticks(rotation=90)
    plt.tight_layout()

**CONCLUSÃO FINAL:** <br>
_A análise da evolução das vendas mensais por categoria entre 2022 e 2025 mostra um comportamento relativamente estável ao longo do tempo, com oscilações pontuais que sugerem sazonalidade moderada, especialmente quedas recorrentes no início do ano (principalmente em janeiro) e picos em meses intermediários ou próximos ao segundo semestre. Categorias como MÓVEIS, ELETRÔNICOS, VESTUÁRIO e PET SHOP apresentam os maiores volumes absolutos e maior volatilidade, indicando maior sensibilidade a ciclos econômicos e períodos promocionais, enquanto PAPELARIA, ESPORTES e BEBIDAS mantêm variações mais suaves e previsíveis. No geral, não há rupturas bruscas de tendência, o que sugere crescimento consistente e maturidade do portfólio, com oportunidades claras para ações sazonais direcionadas às categorias de maior ticket e maior amplitude de variação._

### Pergunta de negócio 02 - **_Quais os top 10 produtos por valor_total e sua participação no total?_**  

> **CONSULTA:** Ranking por valor_total com cálculo de participação (%) sobre o total geral.

> **INSIGHT ESPERADO:** Concentração de receita e dependência de poucos produtos.

In [0]:
from pyspark.sql.functions import sum, col, desc, row_number, round
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

fato_vendas = "dataexperts.gold.fato_vendas"
dim_categoria = "dataexperts.gold.dim_categoria_produto"
dim_produto = "dataexperts.gold.dim_produto"

# Tabelas
produtos = spark.table(dim_produto).select("sk_produto", "produto_id")
categorias = spark.table(dim_categoria).select("sk_categoria", "categoria_nome")

vendas_categoria = spark.table(fato_vendas).groupBy("sk_categoria").agg(sum("valor_total").alias("vendas_categoria"))
vendas_produtos = spark.table(fato_vendas).groupBy("sk_produto", "sk_categoria").agg(sum("valor_total").alias("vendas_produto"))

vendas_totais = vendas_categoria.agg(sum("vendas_categoria")).first()[0]

# Join para saber o id dos produtos e o nome das categorias
vendas_categoria = vendas_categoria.join(categorias.select("sk_categoria", "categoria_nome"), on="sk_categoria").drop("sk_categoria")

vendas_produtos = vendas_produtos.join(categorias.select("sk_categoria", "categoria_nome"), on="sk_categoria").drop("sk_categoria")
vendas_produtos = vendas_produtos.join(produtos.select("sk_produto", "produto_id"), on="sk_produto").drop("sk_produto")


# VENDAS POR PRODUTO (2022~2025)
produtos_rank = vendas_produtos.withColumn("rank", row_number().over(Window.orderBy(col("vendas_produto").desc())))

top_10 = produtos_rank.filter(col("rank") <= 10)
x = top_10.withColumn("participacao [%]", round((col("vendas_produto") * 100) / vendas_totais, 4) )

print("VENDAS POR PRODUTOS")
print("Número de categorias: ", categorias.count())
print("Número de produtos: ", produtos.count())

y = x.agg(sum("participacao [%]").alias("participacao [%]")).first()["participacao [%]"]
print(f"Top 10 produtos representam: {y}%")

x.show()

# VENDAS POR CATEGORIA (2022~2025)
categorias_rank = (
    vendas_categoria
        .withColumn("rank", row_number().over(Window.orderBy(col("vendas_categoria").desc())))
)

top_10 = categorias_rank.filter(col("rank") <= 10)
x = top_10.withColumn("participacao [%]", round((col("vendas_categoria") * 100) / vendas_totais, 4) )
print("VENDAS POR CATEGORIA")
x.show()




**CONCLUSÃO FINAL:** <br>

_Os resultados mostram que os 10 produtos com maior valor_total concentram apenas cerca de 12,22% da receita, indicando baixa dependência de poucos produtos e um portfólio bem distribuído em nível de SKU. Nenhum produto individual ultrapassa 3% de participação, o que reduz riscos operacionais associados a quedas de demanda específicas; porém, a análise por categoria revela maior concentração, com MÓVEIS e ELETRÔNICOS liderando o faturamento, sinalizando que a atenção estratégica deve estar mais voltada à gestão das categorias do que a produtos isolados._

### Pergunta de negócio 03 - **_Quais localidades (UF/cidade) apresentam maior ticket médio?_**

> **CONSULTA:** Média de valor_total por pedido (ou por cliente) agrupada por UF/cidade.

> **INSIGHT ESPERADO:** Potenciais mercados premium e oportunides regionais.

In [0]:
from pyspark.sql.functions import sum, col, desc, row_number, round
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

dim_localidade = "dataexperts.gold.dim_localidade"
fato_vendas = "dataexperts.gold.fato_vendas"

localidades = spark.table(dim_localidade).select("cidade_venda", "estado_venda", "sk_localidade")

vendas_parciais = (spark.table(fato_vendas)
                    .filter(col("sk_localidade").isNotNull())
                    .groupBy("sk_localidade")
                    .agg(sum("valor_total").alias("total_vendas_localidade"))  
                  )

vendas_parciais = vendas_parciais.join(localidades, on="sk_localidade", how="inner").drop("sk_localidade")

vendas_totais = vendas_parciais.agg(sum("total_vendas_localidade").alias("vendas")).first()["vendas"]

media_vendas = vendas_parciais\
                  .withColumn("media_vendas", round(col("total_vendas_localidade")/vendas_totais,6))\
                  .withColumn("participacao[%]", round((col("total_vendas_localidade")*100)/vendas_totais,6))

maiores_tickets = (
    media_vendas
        .withColumn("rank", row_number().over(Window.orderBy(col("media_vendas").desc())))
)

maiores_tickets_localidade = maiores_tickets.filter(col("rank") <= 20)

#maiores_tickets_localidade.show()

estados = media_vendas.groupBy("estado_venda").agg(sum("participacao[%]"))
estados = estados.withColumn("participacao[%]", round(col("sum(participacao[%])"),6)).drop("sum(participacao[%])")
estados_tickets = estados.withColumn("rank", row_number().over(Window.orderBy(col("participacao[%]").desc())))
display(estados_tickets)

sudeste= (
    estados_tickets
    .filter(col("estado_venda").isin(["SP", "RJ", "MG"]))
    .agg(sum(col("participacao[%]")))
).collect()[0]

sul = (
    estados_tickets
    .filter(col("estado_venda").isin(["PR", "SC", "RS"]))
    .agg(sum(col("participacao[%]")))
).collect()[0]

centro_oeste = (
    estados_tickets
    .filter(col("estado_venda").isin(["MT","GO","MS","DF"]))
    .agg(sum(col("participacao[%]")))
).collect()[0]

norte = (
    estados_tickets
    .filter(col("estado_venda").isin(["AC","AP","AM","PA","RO","RR","TO"]))
    .agg(sum(col("participacao[%]")))
).collect()[0]

nordeste = (
    estados_tickets
    .filter(col("estado_venda").isin(["CE","MA","PI","RN","PE","PB","AL","SE","BA"]))
    .agg(sum(col("participacao[%]")))
).collect()[0]

print(f"Sudeste: {sudeste}")
print(f"Sul: {sul}")
print(f"Centro-Oeste: {centro_oeste}")
print(f"Norte: {norte}")
print(f"Nordeste: {nordeste}")



**CONCLUSÃO FINAL:** <br>
_A análise de ticket médio por localidade indica que o Sudeste concentra os mercados mais premium, com destaque absoluto para São Paulo (12,65%) e Rio de Janeiro (8,04%), que juntos representam a maior parcela do valor médio movimentado, refletindo maior poder de consumo e potencial de rentabilidade. Em seguida, estados do Sul (SC, PR, RS) e Nordeste (BA, PE, CE) também se destacam, sugerindo oportunidades regionais relevantes fora do eixo tradicional, especialmente em capitais e polos econômicos. Já as regiões Norte e Centro-Oeste apresentam menor participação relativa, indicando mercados ainda em desenvolvimento ou com menor ticket médio, que podem ser explorados com estratégias de expansão, segmentação de ofertas e amadurecimento do mix de produtos._

### Pergunta de Negócio 04 - **_Como a quantidade média por transação varia por categoria e por ano?_**

A análise da quantidade média de itens por transação, segmentada por categoria e por ano, permite avaliar mudanças no comportamento de compra dos clientes, especialmente no que se refere à formação de cestas menores ou maiores ao longo do tempo.

De forma geral, os dados indicam um comportamento extremamente estável entre 2022 e 2025. Em praticamente todas as categorias, a quantidade média por transação permanece concentrada em torno de 1,84 a 1,86 itens, com variações muito sutis ao longo dos anos. Essa estabilidade sugere que, no período analisado, não houve mudanças relevantes no padrão de compra em termos de volume por pedido, como migração significativa para compras em maior quantidade ou redução expressiva do tamanho das cestas.

As categorias de Alimentos, Bebidas, Calçados, Cosméticos, Eletrodomésticos, Eletrônicos, Papelaria e Vestuário apresentam médias praticamente constantes em todos os anos, reforçando um padrão de consumo recorrente, no qual os clientes tendem a adquirir um número semelhante de itens por transação, independentemente do ano. Esse comportamento é típico de categorias com consumo previsível ou compras mais pontuais, em que a decisão de compra está mais associada à reposição ou à necessidade específica do que à formação de grandes volumes por pedido.

Em categorias como Brinquedos e Pet Shop, observa-se uma leve tendência de redução da quantidade média por transação em 2025, ainda que a variação seja pequena. Esse movimento pode indicar uma mudança marginal no comportamento do consumidor, com maior frequência de compras unitárias ou pedidos mais fracionados, possivelmente influenciados por maior conveniência, promoções focadas em itens específicos ou ajustes no orçamento do consumidor.

Já categorias como Alimentos, Bebidas, Calçados e Pet Shop apresentam um pequeno aumento em 2024, atingindo médias de 1,86 em alguns casos. Embora sutil, esse movimento pode sugerir uma tendência pontual de cestas ligeiramente maiores, possivelmente associada a estratégias promocionais, inflação de preços que incentiva compras antecipadas ou mudanças temporárias no hábito de consumo naquele ano.

No caso de Móveis, a quantidade média por transação permanece consistentemente entre as menores do conjunto, o que é coerente com a natureza da categoria, caracterizada por compras mais unitárias e de maior valor agregado, onde a aquisição de múltiplos itens em uma única transação é menos frequente.

Em síntese, os dados indicam que não há evidência de mudanças estruturais no tamanho médio das compras por transação entre 2022 e 2025. O comportamento do consumidor permanece estável, com cestas predominantemente pequenas e pouca variação entre categorias e anos. Esse insight sugere que estratégias voltadas para aumento de ticket médio por volume, como bundles, kits promocionais ou descontos progressivos por quantidade, ainda representam uma oportunidade relevante para estimular compras com maior número de itens por pedido, uma vez que o padrão atual indica espaço para crescimento nesse aspecto.

In [0]:
from pyspark.sql.functions import avg, col, round
from pyspark.sql.window import Window
import matplotlib.pyplot as plt

fato_vendas = "data_ex.gold.fato_vendas"
dim_categoria = "data_ex.gold.dim_categoria_produto"

vendas = (
    spark.table(fato_vendas)
         .select("quantidade", "sk_categoria", "ano")
)

categorias = (
    spark.table(dim_categoria)
         .select("sk_categoria", "categoria_nome")
)

anos = [2022, 2023, 2024, 2025]

# Join de vendas e categorias
base_analitica = (
    vendas
        .filter(col("ano").isin(anos))
        .join(categorias, on="sk_categoria", how="inner")
)

# Calculando a média por categoria e ano
media_quantidade_categoria_ano = (
    base_analitica
        .groupBy("categoria_nome", "ano")
        .agg(
            round(avg("quantidade"), 2)
            .alias("media_quantidade_por_transacao")
        )
        .orderBy("categoria_nome", "ano")
)

# Transformando em tabela mais compreensível
media_quantidade_pivot = (
    media_quantidade_categoria_ano
        .groupBy("categoria_nome")
        .pivot("ano")
        .agg(round(avg("media_quantidade_por_transacao"), 2))
        .orderBy("categoria_nome")
)

# Visualização em tabelas
media_quantidade_pivot.show()

**CONCLUSÃO FINAL:** <br>
_Os resultados mostram que a quantidade média por transação permanece extremamente estável ao longo dos anos (2022–2025) em todas as categorias, com valores muito próximos de 1,84 a 1,87 itens por compra, indicando ausência de mudanças relevantes no comportamento de compra em termos de volume por pedido. Essa estabilidade sugere que não houve tendência clara de aumento ou redução de compras em packs maiores ou menores, independentemente da categoria, reforçando um padrão consistente de consumo ao longo do tempo e apontando que eventuais variações de receita estão mais associadas a preço, mix de produtos ou frequência de compra do que ao aumento da quantidade adquirida por transação._

###  Pergunta de Negócio 05 - **_Quais clientes são responsáveis por 80% do faturamento (curva ABC)?_**

> **CONSULTA:** Cálculo de valor_total por cliente, ordenação e cumulativo até 80%. 

> **INSIGHT ESPERADO:** Segmentação de clientes estratégicos para ações de retenção

In [0]:
from pyspark.sql.functions import sum, col, desc, row_number, round, when
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

# Importação das tabelas
fato_vendas = "dataexperts.gold.fato_vendas"
dim_clientes = "dataexperts.gold.dim_cliente"

vendas = spark.table(fato_vendas).groupBy("sk_cliente").agg(sum("valor_total").alias("valor_total"))
clientes = spark.table(dim_clientes).select("sk_cliente", "nome_cliente")
vendas = vendas.join(clientes.select("sk_cliente", "nome_cliente"), on="sk_cliente", how="inner").drop("sk_cliente")

# Calculos
vendas_totais = vendas.agg(sum("valor_total")).first()[0]

vendas = vendas.withColumn("porcento", (col("valor_total")*100/vendas_totais))
vendas = vendas.orderBy(col("porcento").desc())

window = Window.orderBy(desc("valor_total")).rowsBetween(Window.unboundedPreceding, 0)

vendas = vendas.withColumn("percent_acumulado", sum("porcento").over(window))

vendas = vendas.withColumn("abc", when(col("percent_acumulado")<= 80, "A")\
                                    .when((col("percent_acumulado")>80) & (col("percent_acumulado")<=95), "B")\
                                    .otherwise("C"))

vendas.show()



In [0]:

# Grafico de Pareto
fig, ax1 = plt.subplots(figsize=(12,6))
df = vendas.toPandas()

# Dominio, clientes 
ax1.bar(df['nome_cliente'], df['valor_total'], color='skyblue')

# Imagem
ax1.set_ylabel('Total Vendas', color='blue')

# Linha do percentual acumulado
ax2 = ax1.twinx()
ax2.plot(df['nome_cliente'], df['percent_acumulado'], color='red', marker='o', linewidth=1)
ax2.set_ylabel('Percentual Acumulado (%)', color='red')

#Limites ABC
ax2.axhline(80, color='green', linestyle='--', label='A = 80%')
ax2.axhline(95, color='orange', linestyle='--', label='B = 95%')

# Infos
ax2.legend(loc='upper left')
plt.title('Curva ABC de Clientes')
plt.show()

In [0]:

plt.figure(figsize=(12,6))
plt.scatter(df['nome_cliente'], df['valor_total'])
#plt.xlabel('Cliente')
#step = max(1, int(len(df) * 0.1))  # mostra a cada 10% dos dados
#plt.xticks(df['nome_cliente'][::step], df['nome_cliente'][::step], rotation=45)
plt.xticks([])
plt.ylabel('Total Vendas')
plt.title('Curva ABC - Scatter')
plt.show()

In [0]:

colors = df['abc'].map({'A':'skyblue','B':'orange','C':'gray'})
plt.figure(figsize=(12,6))
plt.bar(df['nome_cliente'], df['valor_total'], color=colors)
plt.xlabel('Cliente')
plt.ylabel('Total Vendas')
plt.title('Curva ABC - Barras coloridas por Classe')
plt.show()

In [0]:

plt.figure(figsize=(12,6))
plt.plot(df['nome_cliente'], df['percent_acumulado'], marker='o', color='red')
plt.axhline(80, color='green', linestyle='--')
plt.axhline(95, color='orange', linestyle='--')
plt.xlabel('Cliente')
plt.ylabel('Percentual Acumulado (%)')
plt.title('Curva ABC - Percentual Acumulado')
plt.show()

**CONCLUSÃO FINAL:** <br>
_A análise da curva ABC de clientes mostra que o faturamento é fortemente concentrado em um grupo relativamente pequeno de clientes, classificados como classe A, que juntos são responsáveis por 80% da receita total. Os gráficos evidenciam uma curva acumulada quase linear no início, com clientes apresentando valores de compra muito próximos entre si, o que indica que não há um único cliente dominante, mas sim um conjunto consistente de clientes estratégicos com alto valor agregado. Esse perfil reforça a importância de ações focadas de retenção, relacionamento e fidelização para os clientes da classe A, pois pequenas perdas nesse grupo podem gerar impactos significativos no faturamento, enquanto clientes das classes B e C representam oportunidades secundárias de crescimento incremental._

# **ETAPA 6 - AGENTE DE IA PARA ANÁLISES AD-HOC**

---
---

<br>

Essa etapa será responsável por criar uma versão de um modelo de Agente de IA criado manualmente pela plataforma do databricks, carregá-lo com os dados que tivemos como resultado nas etapas anteriores e então proporcionar outras possíveis análises ad-hoc's que possam surgir.

*`Complete as informações necessárias nos trechos que estão destacados em vermelho assim como esse, seguindo o padrão snake_case.`*

<br><br>

---
---

### Parte 1 - **Criação do modelo do agente**

Para criação e utilização do Agente, é necessário pela plaformar seguir os passo:
1. Na aba lateral clicar em _Experiments_
2. Clicar em _GenAI apps & agents_

In [0]:


# =========================================================
# DATAEXPERTS — CRIAÇÃO DO NL → SQL AGENT (SOMENTE FATO)
# =========================================================

import mlflow
from mlflow.pyfunc import PythonModel
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec
from mlflow.deployments import get_deploy_client

# ---------------------------------------------------------
# CONFIGURAÇÕES
# ---------------------------------------------------------
EXPERIMENT_ID = "4074425516892918"
MODEL_NAME = "dataex_sql_agent"
LLM_ENDPOINT = "databricks-llama-4-maverick"

CATALOG = "dataexperts"
SCHEMA = "gold"
FACT_TABLE = "fato_vendas"

mlflow.set_experiment(experiment_id=EXPERIMENT_ID)

# ---------------------------------------------------------
# ASSINATURA DO MODELO
# ---------------------------------------------------------
input_schema = Schema([
    ColSpec("string", "question")
])

output_schema = Schema([
    ColSpec("string", "sql"),
    ColSpec("string", "answer")
])

signature = ModelSignature(inputs=input_schema, outputs=output_schema)

input_example = {
    "question": "Qual foi o faturamento total por mês em 2024?"
}

# ---------------------------------------------------------
# CLIENTE LLM
# ---------------------------------------------------------
client = get_deploy_client("databricks")

# ---------------------------------------------------------
# AGENTE NL → SQL
# ---------------------------------------------------------
class DataExSQLAgent(PythonModel):

    @mlflow.trace
    def predict(self, context, model_input):
        question = model_input["question"]

        system_prompt = f"""
Você é um agente especialista em SQL no Databricks, focado em consultas ad-hoc de análise de dados.
Tabela disponível: {CATALOG}.{SCHEMA}.{FACT_TABLE}
Colunas existentes:
- sk_data
- venda_id
- categoria_id
- quantidade
- preco_lista
- valor_total
- _source_file
- _ingestion_date
- sk_cliente
- sk_produto
- sk_localidade
- ano
- mes

Regras obrigatórias:
- Responda APENAS com a consulta SQL correspondente, sem explicações ou texto adicional.
- Use SOMENTE a tabela acima (não utilize outras tabelas).
- Para faturamento, use SUM(valor_total) como total de vendas.
- Para quantidade total, use SUM(quantidade).
- Para contagem de vendas, use COUNT(venda_id).
- Para preço médio, use AVG(preco_lista).
- Para filtros de data, use as colunas ano e mes (ex: ano = 2024 e mes = 6).
- Comece a consulta sempre com SELECT (comando SQL).
- Use aliases claros (por ex. `f` para fato_vendas).
- Nunca crie colunas ou tabelas que não existem no esquema.
- Não use Markdown nem comentários, retorne apenas o SQL final.
"""


        user_prompt = f"""
Pergunta:
{question}

Retorne apenas o SQL.
"""

        response = client.predict(
            endpoint=LLM_ENDPOINT,
            inputs={
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                "temperature": 0.0
            }
        )

        sql = response["choices"][0]["message"]["content"].strip()

        if context is None:
            return {"sql": sql, "answer": ""}

        df = spark.sql(sql)

        return {
            "sql": sql,
            "answer": df.toPandas().to_json(orient="records")
        }

# ---------------------------------------------------------
# LOG DO MODELO
# ---------------------------------------------------------
with mlflow.start_run(run_name="dataex-sql-agent-fato-only"):
    mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model=DataExSQLAgent(),
        registered_model_name=MODEL_NAME,
        signature=signature,
        input_example=input_example
    )

### Parte 2 - **Execução do modelo do agente com a pergunta personalizada**

In [0]:
# =========================================================
# DATAEXPERTS — USO DO NL → SQL AGENT (SOMENTE FATO)
# =========================================================

import mlflow

# ---------------------------------------------------------
# CONFIGURAÇÕES
# ---------------------------------------------------------
MODEL_NAME = "dataex_sql_agent"

# Unity Catalog → SEMPRE usar alias
MODEL_URI = f"models:/{MODEL_NAME}@somente_tabela_fato_venda"

# ---------------------------------------------------------
# CARREGAR O AGENT
# ---------------------------------------------------------
agent = mlflow.pyfunc.load_model(MODEL_URI)

print(f"✅ Agent carregado: {MODEL_URI}")

# ---------------------------------------------------------
# PERGUNTA
# ---------------------------------------------------------
pergunta = ""

resultado = agent.predict({
    "question": pergunta
})

# ---------------------------------------------------------
# RESULTADOS
# ---------------------------------------------------------
print("SQL GERADO:")
print(resultado["sql"])

print("\nRESPOSTA:")
print(resultado["answer"])


### Parte 3 - **Mostrando resultado da pergunta personalizada**

In [0]:
# =========================================================
# DATAEXPERTS — EXECUÇÃO AUTOMÁTICA DO SQL GERADO PELO AGENT
# =========================================================

sql_gerado = resultado["sql"]

print("🚀 Executando SQL gerado pelo agente:")
print(sql_gerado)

# Executa no Spark
df_resultado = spark.sql(sql_gerado)

# Exibe como tabela no Databricks
display(df_resultado)
